<h2><font color="#004D7F" size=6>Módulo 3. Preprocesamiento de datos</font></h2>

<h1><font color="#004D7F" size=5>3. Métodos de remuestreo</font></h1>

---

<h2><font color="#004D7F" size=5>Índice</font></h2>
<a id="indice"></a>

* [1. Introducción](#section1)
    * [1.1. Librerías y CSV](#section11)
* [2. Validación cruzada](#section2)
    * [2.1. _k_-fold Cross Validation](#section21)
    * [2.2. Validación cruzada repetida](#section22)
    * [2.3. Validación cruzada dejando uno fuera](#section23)
* [3. División en porcentaje](#section3)
    * [3.1. División en porcentaje train/test](#section31)
    * [3.2. División train/test repetidos aleatoriamente](#section32)
* [4. Qué técnica usar](#section4)

In [ ]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

---
<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>

La evaluación es una estimación que podemos usar para hablar sobre qué tan bien creemos que el algoritmo realmente puede funcionar en la práctica. No es una garantía de rendimiento. Una vez que estimamos el rendimiento de nuestro algoritmo, podemos volver a entrenar el algoritmo final en todo el conjunto de datos de entrenamiento y prepararlo para su uso operativo. A continuación, veremos cuatro técnicas diferentes que podemos usar para dividir nuestro conjunto de datos de entrenamiento y crear estimaciones útiles de rendimiento para nuestros algoritmos de Machine Learning:
* Cómo dividir un conjunto de datos en subconjuntos por porcentaje para entrenamiento/validación.
* Cómo evaluar la robustez del modelo utilizando la validación cruzada, k-fold, con y sin repeticiones.
* Cómo evaluar la robustez del modelo usando una validación cruzada dejando uno fuera (LOOCV).
* División en train/test repetidos aleatoriamente.

<a id="section11"></a>
## <font color="#004D7F"> 1.1. Librerías y CSV</font>

Como siempre cargamos el CSV que vamos a utilizar. Así mismo, vamos a cargar las librerías principales que utilizaremos en esta sección.

In [1]:
from google.colab import files
import pandas as pd
import numpy as np

In [2]:
uploaded = files.upload()

Saving pima-indians-diabetes.data.csv to pima-indians-diabetes.data.csv


In [5]:
filename = "pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(filename, names=names)
print(dataframe.shape)
array = dataframe.values
# separate array into input and output components
X = array[:,0:8]
Y = array[:,8]

(768, 9)


<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section2"></a>
# <font color="#004D7F"> 2. Validación cruzada</font>

La validación cruzada es un proceso en la que se realizan $K$ particiones o **folds** de la base de datos y con ellos se realizan $K$ evaluaciones diferentes, de tal forma que todos los casos por lo menos una vez se encuentran en el conjunto de test. Básicamente en la evaluación $i$, la partición $i$ son los casos de test y el resto son los casos de entrenamiento. Finalmente, se realiza una media de los resultados obtenidos en las diferentes evaluaciones. En la siguiente imagen se ve un ejemplo de esto.

<img src="https://static.oschina.net/uploads/img/201609/26155106_OfXx.png" alt="cross-validation" width="500">

<a id="section21"></a>
## <font color="#004D7F"> 2.1. _k_-fold Cross Validation</font>

El método de validación cruzada de _k-fold_ implica dividir el conjunto de datos en _k_-particiones, también llamados _fold_. Cada subconjunto se mantiene mientras el modelo se entrena en todos los demás particiones. Este proceso se repite hasta que se determina la precisión para cada instancia en el conjunto de datos, y se proporciona una estimación de precisión general. Es un método robusto para estimar la precisión, y el tamaño de _k_ puede ajustar la cantidad de sesgo en la estimación, con valores populares establecidos en 5 y 10. 

Puede ver que informamos tanto la media como la desviación estándar de la medida de rendimiento. Al resumir las medidas de rendimiento, es una buena práctica resumir la distribución de las medidas, en este caso suponiendo una distribución gaussiana del rendimiento (un supuesto muy razonable) y registrando la desviación estándar y media.

In [ ]:
# Evaluate using Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

num_folds = 10 #k
seed = 7 # No sirve, tendría que poner luego 'shuffle=True, random_state=seed', pero sale un resultado distinto
kfold = KFold(n_splits=num_folds)
model = LogisticRegression(solver='lbfgs', max_iter=1000)

results = cross_val_score(model, X, Y, cv=kfold)
print(f'Accuracy: {results.mean()*100.0:,.2f}% ({results.std()*100.0:,.2f}%)')

Accuracy: 77.60% (5.16%)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de Scikit-Learn sobre la clase [`KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html?highlight=kfold#sklearn.model_selection.KFold).
</div>

<a id="section22"></a>
## <font color="#004D7F"> 2.2. Validación cruzada repetida</font>

Una extensión de esta técnica de entrenamiento/validación es el poder repetir varias veces el proceso de dividir los datos _k-fold_. En este caso, la precisión final del modelo se toma como la media del número de repeticiones.

Al igual que el caso anterior podemos observar que nos informa tanto la media como la desviación estándar de la medida de rendimiento.

In [ ]:
# Evaluate using Leave One Out Cross Validation
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

num_folds = 10 
seed = 7
num_repeated = 5
repeatedkfold = RepeatedKFold(n_splits=num_folds, n_repeats=num_repeated, random_state=seed)
model = LogisticRegression(solver='lbfgs', max_iter=1000)

results = cross_val_score(model, X, Y, cv=repeatedkfold)
print(f'Accuracy: {results.mean()*100.0:,.2f}% ({results.std()*100.0:,.2f}%)')

Accuracy: 77.56% (4.22%)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de Scikit-Learn sobre la clase [`RepeatedKFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html?highlight=kfold#sklearn.model_selection.RepeatedKFold).
</div>

<a id="section23"></a>
## <font color="#004D7F"> 2.3. Validación cruzada dejando uno fuera</font>

En Validación cruzada dejando uno fuera (Leave One Out Cross Validation, LOOCV), se omite una instancia de datos y se construye un modelo en todas las demás instancias de datos en el conjunto de entrenamiento, repitiéndose este proceso para todas las instancias de datos. El resultado es una gran cantidad de medidas de rendimiento que se pueden resumir en un esfuerzo por proporcionar una estimación más razonable del _Accuracy_ de su modelo. 

Puede ver en la desviación estándar que la puntuación tiene más varianza que los resultados de validación cruzada _k-fold_ descritos anteriormente.

In [ ]:
# Evaluate using Leave One Out Cross Validation
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

loocv = LeaveOneOut()
model = LogisticRegression(solver='lbfgs', max_iter=1000)

results = cross_val_score(model, X, Y, cv=loocv)
print(f'Accuracy: {results.mean()*100.0:,.2f}% ({results.std()*100.0:,.2f}%)')

Accuracy: 77.60% (41.69%)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de Scikit-Learn sobre la clase [`LeaveOneOut`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html?highlight=leaveoneout#sklearn.model_selection.LeaveOneOut).
</div>

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section3"></a>
# <font color="#004D7F"> 3. División en porcentaje</font>

<a id="section31"></a>
## <font color="#004D7F"> 3.1. División en porcentaje train/test</font>

La división de datos implica la partición de los datos en un conjunto de datos de entrenamiento explícito utilizado para preparar el modelo y un conjunto de datos de validación invisible (se dice validación invisible ya que aunque se conozca el atributo clase el modelo lo omitirá para una vez realizada la predicción compararlo para ver si ha acertado) que se utiliza para evaluar el rendimiento del modelo en datos no etiquetados. 

En el ejemplo a continuación, dividimos el conjunto de datos de los indios Pima en divisiones de 67%/33% para entrenamiento y prueba y evaluar el Accuracy de un modelo Logistic Regression.

In [ ]:
# Evaluate using a train and a test set
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

test_size = 0.33
seed = 7 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Crear el modelo
model = LogisticRegression(solver='lbfgs', max_iter=1000)
# Entrenar el modelo
model.fit(X_train, Y_train) 
# Comprobar la eficacia de mi modelo
results = model.score(X_test, Y_test)
print(f'Accuracy: {results.mean()*100.0:,.2f}%')

Accuracy: 78.74%


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de Scikit-Learn sobre la clase [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test_split#sklearn.model_selection.train_test_split).
</div>

<a id="section32"></a>
## <font color="#004D7F"> 3.2. División train/test repetidos aleatoriamente</font>

Otra variación en la validación cruzada de k-fold es crear una división aleatoria de los datos como la división de train/test descrita anteriormente, pero repetir el proceso de división y evaluación del algoritmo varias veces, como la validación cruzada. El siguiente ejemplo divide los datos en una división de train/test del 67%/33% y repite el proceso 10 veces.

Podemos ver que en este caso la distribución de la medida de desempeño está a la par con _k_-fold cross validation pero reducimos la varianza considerablemente.

In [ ]:
# Evaluate using Shuffle Split Cross Validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

test_size = 0.33
seed = 7 
n_splits = 10
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)

# Crear el modelo
model = LogisticRegression(solver='lbfgs', max_iter=1000)
results = cross_val_score(model, X, Y, cv=kfold)
print(f'Accuracy: {results.mean()*100.0:,.2f}% {results.std()*100.0:,.2f}%')

Accuracy: 76.54% 2.24%


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Puede obtener más información en la documentación oficial de Scikit-Learn sobre la clase [`ShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html?highlight=shufflesplit#sklearn.model_selection.ShuffleSplit).
</div>

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section4"></a>
# <font color="#004D7F"> 4. Qué técnica usar</font>

Esta sección enumera algunos consejos para considerar qué técnica de remuestreo usar en diferentes circunstancias.
* Por lo general, la validación cruzada de _k-fold_ es el estándar defacto para evaluar el rendimiento de un algoritmo en datos no etiquetados con _k_ configurado en 3, 5 o 10.
* El uso de una división de train/test es buena para la velocidad cuando se usa un algoritmo lento y produce estimaciones de rendimiento con un sesgo más bajo cuando se usan conjuntos de datos grandes.
* Las técnicas como LOOCV y las divisiones aleatorias repetidas pueden ser intermedios útiles cuando se trata de equilibrar la variación en el rendimiento estimado, la velocidad de entrenamiento del modelo y el tamaño del conjunto de datos.

El mejor consejo es experimentar y encontrar una técnica para su problema que sea rápida y produzca estimaciones razonables de rendimiento que pueda usar para tomar decisiones. En caso de duda, utilice la validación cruzada con un _k-fold_ de 10.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>